# Setup 

## Installing ModelScan

In [1]:

!modelscan -v

modelscan, version 0.8.0


2024-09-16 17:20:49.668153: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-16 17:20:50.544419: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#%pip install -q torch==2.0.1
#%pip install -q transformers==4.31.0
%pip install -q scipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torch

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import os
from utils.pytorch_sentiment_model import download_model, predict_sentiment
from utils.pickle_codeinjection import PickleInject, get_payload

%env TOKENIZERS_PARALLELISM=false

C:\Users\simant.asawale\Desktop\ProtectAI\modelscan\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


env: TOKENIZERS_PARALLELISM=false


# Saving Model


The BERT based sentiment analysis PyTorch model used in the notebook can be found at https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment. The safe model is saved at `./PyTorchModels/safe_model.pt`

In [3]:
# Save a model for sentiment analysis
from typing import Final

model_directory: Final[str] = "PyTorchModels"
if not os.path.isdir(model_directory):
    os.mkdir(model_directory)

safe_model_path = os.path.join(model_directory, "safe_model.pt")

download_model(safe_model_path)

C:\Users\simant.asawale\Desktop\ProtectAI\modelscan\myenv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\simant.asawale\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# Safe Model Prediction

In [4]:
sentiment = predict_sentiment(
    "Stock market was bearish today", torch.load(safe_model_path)
)

C:\Users\simant.asawale\AppData\Local\Temp\ipykernel_14200\4036752315.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  "Stock market was bearish today", torch.load(safe_m

The overall sentiment is: negative with a score of: 85.9%


# Scan Safe Model

The scan results include information on the files scanned, and any issues if found. For the safe model scanned, modelscan finds no model serialization attacks, as expected.

In [8]:


#!chcp 65001 
import sys
import io
import os
#os.environ['PYTHONIOENCODING'] = 'utf-8'
!modelscan --path PyTorchModels/safe_model.pt

No settings file detected at C:\Users\simant.asawale\Desktop\ProtectAI\modelscan\notebooks\modelscan-settings.toml. Using defaults. 

Scanning C:\Users\simant.asawale\Desktop\ProtectAI\modelscan\notebooks\PyTorchModels\safe_model.pt:safe_model\data.pkl using modelscan.scanners.PickleUnsafeOpScan model scan

--- Summary ---

 No issues found! ðŸŽ‰

--- Skipped --- 

Total skipped: 207 - run with --show-skipped to see the full list.


2024-09-16 17:25:03.950669: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-16 17:25:04.814936: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Model Serialization Attack

Here malicious code is injected in the safe model to read aws secret keys. The unsafe model is saved at `./PyTorchModels/unsafe_model.pt`

In [9]:
command = "system"
malicious_code = """cat ~/.aws/secrets
    """

unsafe_model_path = os.path.join(model_directory, "unsafe_model.pt")

payload = get_payload(command, malicious_code)
torch.save(
    torch.load(safe_model_path),
    f=unsafe_model_path,
    pickle_module=PickleInject([payload]),
)

C:\Users\simant.asawale\AppData\Local\Temp\ipykernel_14200\2644983710.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(safe_model_path),


# Unsafe Model Prediction

The malicious code injected in the unsafe model gets executed when it is loaded. The aws secret keys are displayed. 

Also, the unsafe model predicts the sentiments just as well as safe model i.e., the code injection attack will not impact the model performance. The unaffected performance of unsafe models makes the ML models an effective attack vector. 


In [10]:
predict_sentiment("Stock market was bearish today", torch.load(unsafe_model_path))

C:\Users\simant.asawale\AppData\Local\Temp\ipykernel_14200\2044647703.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  predict_sentiment("Stock market was bearish today",

The overall sentiment is: negative with a score of: 85.9%


# Scan Unsafe Model

The scan results include information on the files scanned, and any issues if found. In this case, a critical severity level issue is found in the unsafe model scanned. 

modelscan also outlines the found operator(s) and module(s) deemed unsafe. 

In [11]:
!modelscan --path  ./PyTorchModels/unsafe_model.pt

No settings file detected at C:\Users\simant.asawale\Desktop\ProtectAI\modelscan\notebooks\modelscan-settings.toml. Using defaults. 

Scanning C:\Users\simant.asawale\Desktop\ProtectAI\modelscan\notebooks\PyTorchModels\unsafe_model.pt:unsafe_model\data.pkl using modelscan.scanners.PickleUnsafeOpScan model scan

--- Summary ---

Total Issues: 1

Total Issues By Severity:

    - LOW: 0
    - MEDIUM: 0
    - HIGH: 0
    - CRITICAL: 1

--- Issues by Severity ---

--- CRITICAL ---

Unsafe operator found:
  - Severity: CRITICAL
  - Description: Use of unsafe operator 'system' from module 'nt'
  - Source: C:\Users\simant.asawale\Desktop\ProtectAI\modelscan\notebooks\PyTorchModels\unsafe_model.pt:unsafe_model\data.pkl

--- Skipped --- 

Total skipped: 207 - run with --show-skipped to see the full list.


2024-09-16 17:25:29.780723: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-16 17:25:30.651573: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Reporting Format
ModelScan can report scan results in console (default), json, or custom report (to be defined by user in settings-file). For mode details, please see:  ` modelscan -h` 

## JSON Report

For JSON reporting: `modelscan -p ./path-to/file -r json -o output-file-name.json` 

In [11]:
# This will save the scan results in file: pytorch-model-scan-results.json
!modelscan --path  ./PyTorchModels/unsafe_model.pt -r json -o pytorch-model-scan-results.json

No settings file detected at /Users/mehrinkiani/Documents/modelscan/notebooks/modelscan-settings.toml. Using defaults. 

Scanning /Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt:unsafe_model/data.pkl using modelscan.scanners.PickleUnsafeOpScan model scan
{"modelscan_version": "0.5.0", "timestamp": "2024-01-25T17:10:54.306065", 
"input_path": 
"/Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt"
, "total_issues": 1, "summary": {"total_issues_by_severity": {"LOW": 0, 
"MEDIUM": 0, "HIGH": 0, "CRITICAL": 1}}, "issues_by_severity": {"CRITICAL": 
[{"description": "Use of unsafe operator 'system' from module 'posix'", 
"operator": "system", "module": "posix", "source": 
"/Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt:
unsafe_model/data.pkl", "scanner": "modelscan.scanners.PickleUnsafeOpScan"}]}, 
"errors": [], "scanned": {"total_scanned": 1, "scanned_files": 
["/Users/mehrinkiani/Documents/modelscan/no